In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.optimize import minimize

## Problema principal

Buscamos minimizar las cantidades $$K_1=(4b+c-4a)C_1$$ y $$K_2=(4b+c-4a)C2$$ provenientes de la función
$$K(\tau)=K_1+K_2\tau^{1/2}-\tfrac{2}{3\sqrt{3}}(\tau-1)^{3/2}.$$
Para esto, establecemos el problema de minimización $(P)$ dado por
$$\min_{(a, b, c, C_1, C_2)\in X}\ K_1^2+K_2^2$$
donde $X\subset \mathbb{R}^5$ es el espacio de restricciones del problema.

## Problema de optimización

A continuación se define la función _objetivo_ del problema de optimización, seguido de las _constraints_ de dicho problema.

*_Observación_*: por comodidad, en el código se consideró $C_1=A^{-1}, C_2=B^{-1}$ y $C_3=C$.

In [2]:
def objective(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return ((4*b+c-4*a)**2)*(1/(A**2)+1/(B**2))
def constraint1(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 14*a*(b-2*a)**4-1.5*(b**2)*(A**2)-1.5*(b+c-a)*a*b*(A**2)-1.5*a*(b**2)*(B**2)-0.75*(b+c-a)*(A**4)-0.125*b*(A**2)*(B**2)
def constraint2(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 6*a*(b-2*a)**2-1.5*b*(A**2)-0.75*a*(B**2)
def constraint3(x):
    a=x[0]
    b=x[1]
    c=x[2]
    A=x[3]
    B=x[4]
    return 4*a*(b-2*a)**3-0.25*(b+c-a)*(A**2)-3*(a**2)/((b-2*a)*C)-2*(b-2*a)*(B**2)
def constraint4(x):
    return x[1]-2*x[0]

Se da un punto inicial $x_0$ para el problema. Luego, se consideran las cotas que deben satisfacer las variables de entrada para la función objetivo. Por último, se definen las restricciones del problema.

In [3]:
x0=[1, 8, 0.5, 3, 3]

#restricciones
con1={'type':'ineq', 'fun': constraint1}
con2={'type':'ineq', 'fun': constraint2}
con3={'type':'ineq', 'fun': constraint3}
con4={'type':'ineq', 'fun': constraint4}
cons=[con1, con2, con3, con4]

### Resolución

Fijamos $C_3$ suficientemente grande para resolver el problema.

In [4]:
C=10000000
lista=np.linspace(1, 40, 80).tolist()
for j in lista:
    b1=(1, j)
    b=(0.1, None)
    # Cotas de las variables
    bnds=(b, b, b, b1, b1)
    # No definimos tipo de solver
    sol=minimize(objective, x0, bounds=bnds, constraints=cons)
    # Nos aseguramos de que, en efecto, se satisfagan las restricciones
    if constraint1(sol.x)>0 and constraint2(sol.x)>0 and constraint3(sol.x)>0 and constraint4(sol.x)>0:
        print("---------------------------- Nueva iteración ----------------------------")
        print("val(P)=", sol.fun)
        print("Solución del problema: a=%s, b=%s, c=%s, C1=1/%s, C2=1/%s" %tuple(sol.x))
        print("El valor de K1 es ", (4*sol.x[1]+sol.x[2]-4*sol.x[0])*(1/sol.x[3]))
        print("El valor de K2 es ", (4*sol.x[1]+sol.x[2]-4*sol.x[0])*(1/sol.x[4]))

---------------------------- Nueva iteración ----------------------------
val(P)= 62.37647894013029
Solución del problema: a=0.941875142238227, b=4.851893528670542, c=0.10000000000998419, C1=1/2.3059102711604456, C2=1/3.9620253164456605
El valor de K1 es  6.825969658315489
El valor de K2 es  3.97273421762476
---------------------------- Nueva iteración ----------------------------
val(P)= 61.688128504477774
Solución del problema: a=3.7840382656884692, b=19.805659193978197, c=0.1, C1=1/9.353951253541172, C2=1/16.79746835443038
El valor de K1 es  6.861964743386866
El valor de K2 es  3.821199859336514
---------------------------- Nueva iteración ----------------------------
val(P)= 61.682640673709486
Solución del problema: a=3.895049105925121, b=20.386914513422, c=0.10000000000001051, C1=1/9.628383173195356, C2=1/17.29113924050632
El valor de K1 es  6.861739966260796
El valor de K2 es  3.8208854090549176
---------------------------- Nueva iteración ----------------------------
val(P)= 61.

## Conclusiones

Al iterar sobre distintos valores de cotas inferiores (sobre $b$ en particular) se ve un comportamiento similar para $K_1$ y $K_2$, pues mantienen valores similares que no bajan de $6.85$ y $3.81$ respectivamente.

Del mismo modo, se puede apreciar una conclusión análoga al variar la cota inferior de $C_1, C_2$, donde se mantiene $val(P)$ y los valores de $K_1$ y $K_2$.

Por último, mencionar que el problema es altamente sensible a la condición inicial $x_0$ asignada.